In [ ]:
using Piccolo
using Optim
using SparseArrays

# Exercise goals
-----
- Learn the quantum isomorphisms that map variables to real-valued state vectors
- Study how gradient descent and Newton's method can be used to optimize quantum controls. 

# I. Isomorphisms
-----

- The standard quantum states are _kets_, $|\psi\rangle$, and _Unitaries_, $U$.
- Open quantum system require _density matrices_, $\rho$, and _quantum channels_, $\Phi$.
- Standard quantum states have an open system counterpart,

\begin{align}
    \text{closed} &\longrightarrow \text{open}  \\ \hline
    |\psi\rangle &\longrightarrow |\psi\rangle \langle \psi | \\
    U &\longrightarrow U \cdot U^\dagger 
\end{align}

🚧 ⚠️ If you are seeing a lot of boxes like Ũ⃗, it is _very_ useful to have the [JuliaMono](https://juliamono.netlify.app/) fonts for Piccolo. Install and [change the default font family](https://code.visualstudio.com/docs/terminal/appearance).

In [ ]:
ψ = [1; 2] + im * [3; 4]
ψ̃ = ket_to_iso(ψ)

In [ ]:
iso_to_ket(ψ̃)

In [ ]:
# We often need to convert a complex matrix U to a real vector, Ũ⃗. 
U = [1 5; 2 6] + im * [3 7; 4 8]

In [ ]:
Ũ⃗ = operator_to_iso_vec(U)

In [ ]:
iso_vec_to_operator(Ũ⃗)

In [ ]:
# Warning: The isomorphism `density_to_iso_vec` is not the same as `operator_to_iso_vec`.
ρ = [1 2; 3 4] + im * [5 6; 7 8]
ρ̃⃗ = density_to_iso_vec(ρ)

# II. Quantum dynamics
-----

- Let's flip a qubit from the ground state to the excited state.
- Introduce the isomorphisms that make quantum dynamics real-valued.  
- Use [PiccoloQuantumObjects](https://docs.harmoniqs.co/PiccoloQuantumObjects/dev/) to make a quantum system.
- Use a rollout to integrate the quantum system forward in time.

\begin{equation}
    H(u(t)) = \underbrace{u_1(t) XI + u_2(t) YI}_\text{qubit 1} 
    + \underbrace{u_3(t) IX + u_4(t) IY}_\text{qubit 2} + \underbrace{u_5(t) XX}_\text{coupling}
\end{equation}

In [ ]:
H_drives = [
    PAULIS.X ⊗ PAULIS.I,
    PAULIS.Y ⊗ PAULIS.I,
    PAULIS.I ⊗ PAULIS.X,
    PAULIS.I ⊗ PAULIS.Y,
    PAULIS.X ⊗ PAULIS.X
]

system = QuantumSystem(H_drives)

# Entangling gate
U_goal = GATES.CX

# Timing information (e.g. 20 ns superconducting qubit gate)
T = 50
Δt = 0.4

- Quantum systems contain the operators we need

In [ ]:
get_drift(system)

In [ ]:
get_drives(system)

- We can use a system to perform a rollout.

In [ ]:
unitary_rollout(controls, timesteps, system)

In [ ]:
unitary_rollout_fidelity(U_goal, controls, timesteps, system)

In [ ]:
# Piccolo (we'll learn more about this later)
prob = UnitarySmoothPulseProblem(system, U_goal, T, Δt);

# save these initial controls for later
a_init = prob.trajectory.a
series(get_times(prob.trajectory), a_init)

In [ ]:
solve!(prob, max_iter=100, print_level=1, verbose=false)

ℱ = 1 - unitary_rollout_fidelity(prob.trajectory, system)

println("The infidelity is ", 1 - ℱ)

In [ ]:
a_final = prob.trajectory.a
series(get_times(prob.trajectory), a_final)

# III. GRAPE
-----

The [GRAPE algorithm](https://doi.org/10.1016/j.jmr.2004.11.004) comes from NMR in 2004, and there is a [Julia version](https://github.com/JuliaQuantumControl/GRAPE.jl). We'll reproduce GRAPE in this example.

In [ ]:
timesteps = fill(Δt, T)

GRAPE(controls) = unitary_rollout_fidelity(U_goal, controls, timesteps, system)

## Automatic differentiation
- It's quick to test! Compare different algorithms, e.g., `Newton()`, `GradientDescent()`, `LBFGS()`
- If you switch from gradient descent to a quasi-Newton method, you get to [write another paper](https://www.sciencedirect.com/science/article/abs/pii/S1090780711002552).

In [ ]:
result_GRAPE = optimize(GRAPE, a_init, LBFGS())

In [ ]:
# It's hard to tell the difference between the initial value and the converged solution!
series(cumsum(timesteps), result_GRAPE.minimizer)

## Analytic gradients
- We can combine forward and backward rollouts to compute the gradients,
\begin{align}
    \frac{\partial U(T)}{\partial u_k(t)} &= U(T, t) (-i H_k \Delta t) U(t) \\
   \Rightarrow \langle\psi_\text{goal} | \frac{\partial U(T)}{\partial u_k(t)} |\psi_\text{init.}\rangle &= -i \Delta t \langle\psi_\text{goal}^\text{bwd.}(t) | H_k |\psi_\text{init.}^\text{fwd.}(t) \rangle.
\end{align}


In [ ]:
# TODO: Calculate the gradient of a state infidelity using forward-mode autodiff, and compare it to the analytic gradient hinted at here.

# III. Function Spaces
-----

- Pick a function basis for the controls and optimize the coefficients. Some choices are [trig functions](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.84.022326) or [Slepians](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.97.062346). There were [comparative studies](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.84.022305).
- Our optimization parameters are now coefficients of the basis,
\begin{equation}
    u(t) = u_0 + \sum_{j=1}^{n} c_j a_j(t)
\end{equation}
- The modes $a_j(t)$ stay fixed, and the coefficients $c_j$ are optimized.

In [ ]:
# First n=5 entries in a Fourier series, including the constant term
n = 5
fourier_series = [cos.(π * j * (0:T-1) / T .- π/2) for j in 0:n-1]

function get_controls(coefficients)
    a(c) = sum(cⱼ * aⱼ for (cⱼ, aⱼ) in zip(c, fourier_series))
    return stack([a(c) for c in eachrow(coefficients)], dims=1)
end

function CRAB(coefficients)
    controls = get_controls(coefficients)
    return unitary_rollout_fidelity(U_goal, controls, timesteps, system)
end

In [ ]:
Random.seed!(1234)
c_init = rand(system.n_drives, n)
result_CRAB = optimize(CRAB, c_init, LBFGS(); autodiff = :forward)

series(cumsum(timesteps), get_controls(result_CRAB.minimizer))

- These shapes are a lot nicer! But performance depends a lot on the expressivity and initial condition.

In [ ]:
Random.seed!(5678)
c_init = randn(system.n_drives, n)
result_CRAB = optimize(CRAB, c_init, LBFGS(); autodiff = :forward)
series(cumsum(timesteps), get_controls(result_CRAB.minimizer))

## Control filters

- Add spectral filters to the controls

In [ ]:
# TODO: Consider

# IV. States in costs
-----

- Let's switch to a transmon, which has more than two levels and can be _leaky_.

\begin{equation}
H(u(t)) = \tfrac{1}{2} \eta a^\dagger a^\dagger a a + u_1(t) (a + a^\dagger) - i u_2(t) (a - a^\dagger)
\end{equation}

- Add a leakage penalty. Notice that working with states can be awkward.

In [ ]:
a = annihilate(4)
X = a + a'
Y = -im * (a - a')
η = 0.1
H_drift = 1/2 * a'a'a*a
H_drives = [X, Y]
system = QuantumSystem(H_drives)

In [ ]:
# TODO: 
# - Add an L2 penalty to states that are not in the computational basis.
# - Use a modified GRAPE cost to penalize leakage while maintaining fidelity.
# - Study how the leakage changes with the anharmonicity η.